# BMW Sales Trend Forecasting & Alert System

This notebook mirrors the functionality of the simplified `main.py` script. It performs the following steps:
1. Data Loading & Preprocessing
2. Exploratory Data Analysis
3. Time Series Aggregation
4. Visualization (Static & Interactive)
5. Reporting

**Note:** Forecasting (ARIMA) and Alerting features have been removed in this simplified version.

In [ ]:
import sys
import os
import numpy as np
from datetime import datetime

# Ensure local modules can be imported
sys.path.append(os.getcwd())

from config import (
    DATA_CSV_FILE, out_path,
    ENABLE_DATA_PROCESSING, ENABLE_TIME_SERIES, ENABLE_STATIC_PLOTS,
    ENABLE_REPORTING, ENABLE_DASHBOARDS, ENABLE_AGGREGATOR,
    ENABLE_EXPLORATORY_ANALYSIS
)
from utils import print_section, clean_outputs, zip_all_outputs
from data import download_required_files, load_and_explore_data, preprocess_data
from analysis import aggregate_time_series
from exploratory_analysis import exploratory_data_analysis
from viz_static import create_overview_visualizations, create_heatmap
from viz_interactive import create_interactive_dashboard, create_heatmap_interactive
from reporting import generate_monthly_report, generate_final_summary
from aggregator import create_aggregator_html

In [ ]:
# Initialize variables
df = None
df_clean = None
df_yearly = None
ts_data = None
ts_years = None
df_model_yearly = None
df_region_yearly = None

# Clean output directory
clean_outputs()

## 1. Data Loading & Preprocessing

In [ ]:
if ENABLE_DATA_PROCESSING:
    download_required_files()
    df = load_and_explore_data(DATA_CSV_FILE)
    df_clean = preprocess_data(df)
    
    if ENABLE_EXPLORATORY_ANALYSIS:
        exploratory_data_analysis(df_clean)

## 2. Time Series Aggregation

In [ ]:
if ENABLE_TIME_SERIES:
    df_yearly, ts_data, ts_years, df_model_yearly, df_region_yearly = aggregate_time_series(df_clean)

## 3. Static Visualizations

In [ ]:
if ENABLE_STATIC_PLOTS:
    create_overview_visualizations(df_yearly, df_clean)
    create_heatmap(df_clean)

## 4. Reporting

In [ ]:
if ENABLE_REPORTING:
    # Ensure we have necessary data or defaults
    average_sales = df_yearly['Total_Sales'].mean() if df_yearly is not None else 0
    
    # Create dummy data for reporting if missing
    monthly_report = generate_monthly_report(df_clean, average_sales)
    print(monthly_report)
    
    report_filename = out_path(f"sales_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
    with open(report_filename, 'w', encoding='utf-8') as f:
        f.write(monthly_report)
    print(f"\n✅ Saved: {report_filename}")

## 5. Interactive Dashboards

In [ ]:
if ENABLE_DASHBOARDS:
    create_interactive_dashboard(ts_years, ts_data, df_yearly, df_clean)
    create_heatmap_interactive(df_model_yearly)

## 6. Aggregator & Final Summary

In [ ]:
if ENABLE_AGGREGATOR:
    create_aggregator_html()
    zip_all_outputs()

if ENABLE_REPORTING:
    average_sales = df_yearly['Total_Sales'].mean() if df_yearly is not None else 0
    
    # Create dummy data for summary if missing
    if ts_data is None:
        ts_data = np.array([0, 0])
    if ts_years is None:
        ts_years = np.array([2020, 2021])

    generate_final_summary(df_clean, average_sales, ts_years, ts_data)